In [1]:
from pprint import pprint

simplified = {"a":[1,2], "b":[3,4]}

pprint(simplified)

{'a': [1, 2], 'b': [3, 4]}


In [2]:
import rdkit

In [3]:
import copy

In [4]:
pprint(simplified)

{'a': [1, 2], 'b': [3, 4]}


In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [6]:
#import ase # --> get hill chemical formula if necessary
# https://wiki.fysik.dtu.dk/ase/ase/atoms.html#ase.Atoms.get_chemical_formula
# a good sanity check

In [7]:
# Commented out fields that may not be strictly necessary and which .
state_format = {'num_atoms': [], #:int
                'charges': [], #: list(int)
                #'fragments', #: tuple(tuple(int)) (or list(list(int)) ?)
                'positions': [], #: numpy.ndarray
                'rxn': [], # unique ID of elementary reaction
                #'wB97x_6-31G(d).energy', 
                #'wB97x_6-31G(d).atomization_energy', 
                #'wB97x_6-31G(d).forces', 
                #'formula'
               }

In [8]:
raw_dataset_format = {"reactant": copy.deepcopy(state_format), 
                      "transition_state": copy.deepcopy(state_format), 
                      "product": copy.deepcopy(state_format),
                      "single_fragment": [], # 0 or 1
                      "use_ind": [], # subset of indices to use as train data
                     }

In [9]:
import pickle
npz_path = "./oa_reactdiff/data/transition1x/train.pkl"
train_pkl = pickle.load(open(npz_path, "rb"))

In [10]:
temp_array = train_pkl["reactant"]["positions"][0]
temp_array

array([[ 0.25745181, -0.8735503 ,  0.30037837],
       [-0.56172165,  0.02103662,  0.89758269],
       [-0.67397285,  1.12227861,  0.24385745],
       [ 0.12700341,  0.97910814, -0.88440928],
       [ 0.64463946, -0.19480799, -0.80248384],
       [-1.01875544, -0.26059762,  1.83467119],
       [ 1.32874423, -0.67792179, -1.48481407]])

In [11]:
print(type(temp_array))
print(type(temp_array[0]))
print(type(temp_array[0][0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float64'>


In [12]:
train_pkl["reactant"]["charges"][0]

[8, 6, 7, 7, 6, 1, 1]

In [13]:
train_pkl["reactant"]["formula"][0]

'C2H2N2O'

# Labeling fragmentation of product state
04/06/2025:
After preparing SCAN data for the Transition1x.pkl format in the grrm_log parser environment with the notebook `all_repath_SCAN_data_formatting-20250604.ipynb`, we use the OAReactDiff environment to complete the pre-confidence model job and label the instances in SCAN by whether the product state of the i'th instance is more than one fragment or not. While not a perfect emulation, we use RDkit determineconnectivity which matched Transition1x labeling at aroun 94%. 


In [14]:
import pickle
import rdkit
from rdkit import Chem
import torch

In [15]:
atom_num2sym = {
    1: 'H',    2: 'He',   3: 'Li',   4: 'Be',   5: 'B',    6: 'C',    7: 'N',    8: 'O',    9: 'F',    10: 'Ne',
    11: 'Na',  12: 'Mg',  13: 'Al',  14: 'Si',  15: 'P',   16: 'S',   17: 'Cl',  18: 'Ar',  19: 'K',   20: 'Ca',
    21: 'Sc',  22: 'Ti',  23: 'V',   24: 'Cr',  25: 'Mn',  26: 'Fe',  27: 'Co',  28: 'Ni',  29: 'Cu',  30: 'Zn',
    31: 'Ga',  32: 'Ge',  33: 'As',  34: 'Se',  35: 'Br',  36: 'Kr',  37: 'Rb',  38: 'Sr',  39: 'Y',   40: 'Zr',
    41: 'Nb',  42: 'Mo',  43: 'Tc',  44: 'Ru',  45: 'Rh',  46: 'Pd',  47: 'Ag',  48: 'Cd',  49: 'In',  50: 'Sn',
    51: 'Sb',  52: 'Te',  53: 'I',   54: 'Xe',  55: 'Cs',  56: 'Ba',  57: 'La',  58: 'Ce',  59: 'Pr',  60: 'Nd',
    61: 'Pm',  62: 'Sm',  63: 'Eu',  64: 'Gd', 65: 'Tb',  66: 'Dy',  67: 'Ho',  68: 'Er',  69: 'Tm',  70: 'Yb',
    71: 'Lu',  72: 'Hf',  73: 'Ta',  74: 'W',   75: 'Re',  76: 'Os',  77: 'Ir',  78: 'Pt',  79: 'Au',  80: 'Hg',
    81: 'Tl',  82: 'Pb',  83: 'Bi',  84: 'Po',  85: 'At',  86: 'Rn',  87: 'Fr',  88: 'Ra',  89: 'Ac',  90: 'Th',
    91: 'Pa',  92: 'U',   93: 'Np',  94: 'Pu',  95: 'Am',  96: 'Cm',  97: 'Bk',  98: 'Cf',  99: 'Es', 100: 'Fm',
    101: 'Md', 102: 'No', 103: 'Lr', 104: 'Rf', 105: 'Db', 106: 'Sg', 107: 'Bh', 108: 'Hs', 109: 'Mt', 110: 'Ds',
    111: 'Rg', 112: 'Cn', 113: 'Nh', 114: 'Fl', 115: 'Mc', 116: 'Lv', 117: 'Ts', 118: 'Og'
}

atom_sym2num = {v:k for k, v in atom_num2sym.items()}

In [16]:
with open("/scr/trond/SCAN/SCAN_train_wo_product_frag_labels.pkl", "rb") as f:
    scan_pkl = pickle.load(f)

In [17]:
def assemble_xyz(atom_symbols, positions: torch.Tensor, comment_line: str="") -> str:
    """
    Assembles an XYZ string from atom symbols and torch tensor positions.
    Ensures robust formatting of coordinates for RDKit parsing.

    Args:
        atom_symbols (list[str]): List of atomic symbols (e.g., ['C', 'H', 'H']).
        positions (torch.Tensor): A Nx3 torch tensor of atomic coordinates.

    Returns:
        str: The formatted XYZ string.
    """
    num_atoms = len(atom_symbols)
    xyz_lines = [str(num_atoms), comment_line]

    for i in range(num_atoms):
        symbol = atom_symbols[i]
        # Format coordinates to a reasonable number of decimal places (e.g., 6)
        # Using 'f' format ensures fixed-point notation, avoiding 'e-XX' for small numbers.
        # This is CRUCIAL for RDKit's MolFromXYZBlock to parse correctly.
        x = f"{positions[i, 0]:.6f}"
        y = f"{positions[i, 1]:.6f}"
        z = f"{positions[i, 2]:.6f}"
        xyz_lines.append(f"{symbol}  {x}  {y}  {z}")

    return "\n".join(xyz_lines)

In [18]:
def get_fragments(xyz: str, bond_cov_factor: float=1.3, useHueckel: bool=False) -> list:
    mol_block = Chem.MolFromXYZBlock(xyz)
    # useHueckel=False, int charge=0
    Chem.rdDetermineBonds.DetermineConnectivity(mol_block, useHueckel=useHueckel, charge=0, covFactor=bond_cov_factor, useVdw=True)
    #rdkit.Chem.rdDetermineBonds.DetermineConnectivity(mol_block, covFactor=bond_cov_factor, useVdw=True)
    #rdDetermineBonds.DetermineBondOrders(mol_block, charge=0)

    #fragmols = Chem.GetMolFrags(mol_block, asMols=True, sanitizeFrags=False)
    #fragments = []
    return Chem.GetMolFrags(mol_block, asMols=False, sanitizeFrags=False)

In [19]:
from rdkit.Chem.rdDetermineBonds import DetermineConnectivity

In [20]:
i = 777
charges = scan_pkl["product"]["charges"][i]
z = [atom_num2sym[x] for x in charges]
positions = scan_pkl["product"]["positions"][i]
xyz = assemble_xyz(z, torch.tensor(positions))

In [21]:
fragments = get_fragments(xyz, bond_cov_factor=3)
fragments

((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16),)

In [22]:
len(fragments)

1

In [24]:
fragment_fraction = 0
bond_cov_factor = 2.3
dataset_size = 108809

for i in range(dataset_size):
    if i % 1000 == 0:
        print(i)
    charges = scan_pkl["product"]["charges"][i]
    z = [atom_num2sym[x] for x in charges]
    positions = scan_pkl["product"]["positions"][i]
    xyz = assemble_xyz(z, torch.tensor(positions))
    fragments = get_fragments(xyz, bond_cov_factor=bond_cov_factor)
    if len(fragments) > 1:
        fragment_fraction += 1

print(fragment_fraction)
fragment_fraction /= dataset_size
print(fragment_fraction)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
68688
0.6312713102776425


With default covalent radius factor, there would be no training data. Better just ignore this for the initial pass and try more later.